In [1]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate
!pip install transformers
import accelerate
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

In [2]:
!mkdir /content/data

In [3]:
result_df = pd.read_csv("/content/Main.csv")
# result_df = result_df.drop('article_id','' axis=1)
result_df = result_df.sample(frac=1).reset_index(drop=True)

# Assuming 'df' is your DataFrame
result_df['Antagonist'] = result_df['ff'].apply(lambda x: 1 if x == 'Antagonist' else 0)
result_df['Protagonist'] = result_df['ff'].apply(lambda x: 1 if x == 'Protagonist' else 0)
result_df['Innocent'] = result_df['ff'].apply(lambda x: 1 if x == 'Innocent' else 0)

result_df

article_id        dominant_narrative  dominant_subnarrative  \
0    EN_CC_100038.txt                   the Fed                   1081   
1    EN_UA_023008.txt  Davos/UK dominated media                    372   
2    EN_UA_021270.txt                    Yermak                    667   
3    EN_UA_100587.txt                Ukrainians                   2652   
4    EN_UA_011260.txt            European Union                   1075   
..                ...                       ...                    ...   
332  EN_CC_100005.txt         Stopp Oljeletinga                   1700   
333  EN_UA_015443.txt                   Ukraine                    732   
334  EN_UA_000923.txt                        UK                   1581   
335  EN_UA_003579.txt                World Bank                    851   
336  EN_CC_100095.txt            Greta Thunberg                   1691   

     explanation           ff          fff       f  \
0           1087   Antagonist  Incompetent     NaN   
1            395   Antagonist     Deceiver     NaN   
2            672   Antagonist  Incompetent     NaN   
3           2661     Innocent    Exploited  Victim   
4           1088   Antagonist  Conspirator     NaN   
..           ...          ...          ...     ...   
332         1716   Antagonist    Terrorist     NaN   
333          738  Protagonist     Underdog     NaN   
334         1582  Protagonist     Guardian     NaN   
335          860  Protagonist   Peacemaker     NaN   
336         1704   Antagonist     Deceiver     NaN   

                                       article_content  \
0    The Fed’s Climate Studies Are Full Of Hot Air ...   
1    What Everyone Is Missing About the Putin/Carls...   
2    Ukraine's Fate Will Be Decided In Coming Year,...   
3    Almost 300 kids snatched from Ukrainian city i...   
4    Russia has a clear plan to resolve the conflic...   
..                                                 ...   
332  Climate Crazies Fail in Attempt to Vandalize A...   
333  Ukraine War: Kyiv’s allies fear military suppo...   
334  Boris Johnson demands Putin ‘steps back from t...   
335  International agencies call for urgent aid to ...   
336  Climate Change Hoaxer Greta Thunberg Charged f...   

                                                Prompt  Antagonist  \
0     returns hundreds of research papers, press re...           1   
1    three-edged sword: your side, their side, and ...           1   
2    aff for the Office of the President of Ukraine...           1   
3    ers in occupied Ukraine by 2026.\n\nThe Russia...           0   
4     new Ukrainian leadership is trying to purge t...           1   
..                                                 ...         ...   
332  igans, and reported there was some glue residu...           1   
333  s a divisive presidential campaign,” the Finan...           0   
334  nst Russian aggression towards Ukraine, a Cabi...           0   
335  s soaring in recent weeks, while rising natura...           0   
336  e oil tankers.\n\nKjellström added, “After hav...           1   

     Protagonist  Innocent  
0              0         0  
1              0         0  
2              0         0  
3              0         1  
4              0         0  
..           ...       ...  
332            0         0  
333            1         0  
334            1         0  
335            1         0  
336            0         0  

[337 rows x 12 columns]

In [4]:

df = pd.get_dummies(result_df, columns=['fff'])

# Remove prefix from column names
df.columns = df.columns.str.replace('fff_', '')

print(df)

# Convert boolean values to integers (1 and 0)
for col in df.select_dtypes(include='bool').columns:
    df[col] = df[col].astype(int)

df['Prompt'] = 'Given the follwoing sentence: ' + df['Prompt']+ '  and the mentioned entity: ' + df['dominant_narrative'] + ' provide the label or labels for it.'
df

           article_id        dominant_narrative  dominant_subnarrative  \
0    EN_CC_100038.txt                   the Fed                   1081   
1    EN_UA_023008.txt  Davos/UK dominated media                    372   
2    EN_UA_021270.txt                    Yermak                    667   
3    EN_UA_100587.txt                Ukrainians                   2652   
4    EN_UA_011260.txt            European Union                   1075   
..                ...                       ...                    ...   
332  EN_CC_100005.txt         Stopp Oljeletinga                   1700   
333  EN_UA_015443.txt                   Ukraine                    732   
334  EN_UA_000923.txt                        UK                   1581   
335  EN_UA_003579.txt                World Bank                    851   
336  EN_CC_100095.txt            Greta Thunberg                   1691   

     explanation           ff       f  \
0           1087   Antagonist     NaN   
1            395   Antagonist

article_id        dominant_narrative  dominant_subnarrative  \
0    EN_CC_100038.txt                   the Fed                   1081   
1    EN_UA_023008.txt  Davos/UK dominated media                    372   
2    EN_UA_021270.txt                    Yermak                    667   
3    EN_UA_100587.txt                Ukrainians                   2652   
4    EN_UA_011260.txt            European Union                   1075   
..                ...                       ...                    ...   
332  EN_CC_100005.txt         Stopp Oljeletinga                   1700   
333  EN_UA_015443.txt                   Ukraine                    732   
334  EN_UA_000923.txt                        UK                   1581   
335  EN_UA_003579.txt                World Bank                    851   
336  EN_CC_100095.txt            Greta Thunberg                   1691   

     explanation           ff       f  \
0           1087   Antagonist     NaN   
1            395   Antagonist     NaN   
2            672   Antagonist     NaN   
3           2661     Innocent  Victim   
4           1088   Antagonist     NaN   
..           ...          ...     ...   
332         1716   Antagonist     NaN   
333          738  Protagonist     NaN   
334         1582  Protagonist     NaN   
335          860  Protagonist     NaN   
336         1704   Antagonist     NaN   

                                       article_content  \
0    The Fed’s Climate Studies Are Full Of Hot Air ...   
1    What Everyone Is Missing About the Putin/Carls...   
2    Ukraine's Fate Will Be Decided In Coming Year,...   
3    Almost 300 kids snatched from Ukrainian city i...   
4    Russia has a clear plan to resolve the conflic...   
..                                                 ...   
332  Climate Crazies Fail in Attempt to Vandalize A...   
333  Ukraine War: Kyiv’s allies fear military suppo...   
334  Boris Johnson demands Putin ‘steps back from t...   
335  International agencies call for urgent aid to ...   
336  Climate Change Hoaxer Greta Thunberg Charged f...   

                                                Prompt  Antagonist  \
0    Given the follwoing sentence:  returns hundred...           1   
1    Given the follwoing sentence: three-edged swor...           1   
2    Given the follwoing sentence: aff for the Offi...           1   
3    Given the follwoing sentence: ers in occupied ...           0   
4    Given the follwoing sentence:  new Ukrainian l...           1   
..                                                 ...         ...   
332  Given the follwoing sentence: igans, and repor...           1   
333  Given the follwoing sentence: s a divisive pre...           0   
334  Given the follwoing sentence: nst Russian aggr...           0   
335  Given the follwoing sentence: s soaring in rec...           0   
336  Given the follwoing sentence: e oil tankers.\n...           1   

     Protagonist  ...  Rebel  Saboteur  Scapegoat  Spy  Terrorist  Traitor  \
0              0  ...      0         0          0    0          0        0   
1              0  ...      0         0          0    0          0        0   
2              0  ...      0         0          0    0          0        0   
3              0  ...      0         0          0    0          0        0   
4              0  ...      0         0          0    0          0        0   
..           ...  ...    ...       ...        ...  ...        ...      ...   
332            0  ...      0         0          0    0          1        0   
333            1  ...      0         0          0    0          0        0   
334            1  ...      0         0          0    0          0        0   
335            1  ...      0         0          0    0          0        0   
336            0  ...      0         0          0    0          0        0   

     Tyrant  Underdog  Victim  Virtuous  
0         0         0       0         0  
1         0         0       0         0  
2         0         0       

In [5]:
label_df = df[['Prompt', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
# print(label_df['sentence'][0])

In [6]:
train_df = label_df[:300]
validation_df = label_df[300:]
train_labels_df = label_df[:300]
validation_labels_df = label_df[300:]


In [7]:
train_sentences = train_df['Prompt'].values.tolist()
train_labels = train_labels_df.iloc[:, 1:].values
train_labels = train_labels.astype(np.float32)

validation_sentences = validation_df['Prompt'].values.tolist()
validation_labels = validation_labels_df.iloc[:, 1:].values
validation_labels = validation_labels.astype(np.float32)

In [8]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
# model

from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
model

# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-8b-base")
# model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.0-8b-base")
# model


# Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
# model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-large", num_labels=25, problem_type="multi_label_classification")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine


## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [9]:
class ArgDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [10]:
# test_labels_df

In [11]:
train_encodings = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
train_dataset = ArgDataset(train_encodings, train_labels)

validation_encodings = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)
validation_dataset = ArgDataset(validation_encodings, validation_labels)

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import os

label_names = train_labels_df.columns[1:].tolist()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)  # Convert probabilities to binary predictions

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)

    # Generate classification report
    # report = classification_report(labels, preds, target_names=label_names, zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'classification_report': report
    }

os.environ["WANDB_DISABLED"] = "true"

In [13]:
# import torch
# from torch import nn
# from transformers import Trainer

# # Define the Contrastive Loss function
# contrastive_loss_fn = nn.CosineEmbeddingLoss(margin=0.5)

# # Function to create positive and negative pairs based on fine-grained roles
# def create_pairs(embeddings, labels):
#     positive_pairs = []
#     negative_pairs = []

#     # Ensure labels are single scalar values for comparison
#     labels = labels.cpu().numpy()  # Convert to numpy for easier indexing and comparison
#     # print(labels)

#     for i in range(len(labels)):
#         for j in range(i + 1, len(labels)):
#             # Compare scalar values for each label to determine positive/negative pairs
#             p = np.array(labels[i])
#             q = np.array(labels[j])
#             are_equal = np.allclose(p, q)
#             if are_equal:  # Positive pair
#                 positive_pairs.append((embeddings[i], embeddings[j]))
#             else:  # Negative pair
#                 negative_pairs.append((embeddings[i], embeddings[j]))

#     return positive_pairs, negative_pairs


# class ContrastiveTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         # Classification loss
#         labels = inputs.get("labels")
#         outputs = model(**inputs, output_hidden_states=True)
#         classification_loss = outputs.loss

#         # Obtain embeddings from the last hidden state
#         embeddings = outputs.hidden_states[-1][:, 0, :]
#         positive_pairs, negative_pairs = create_pairs(embeddings, labels)

#         contrastive_loss = 0

#         # Calculate contrastive loss for positive pairs
#         if positive_pairs:
#             pos_emb1 = torch.stack([p[0] for p in positive_pairs])
#             pos_emb2 = torch.stack([p[1] for p in positive_pairs])
#             contrastive_loss += contrastive_loss_fn(pos_emb1, pos_emb2, torch.ones(pos_emb1.size(0)).to(pos_emb1.device))

#         # Calculate contrastive loss for negative pairs
#         if negative_pairs:
#             neg_emb1 = torch.stack([n[0] for n in negative_pairs])
#             neg_emb2 = torch.stack([n[1] for n in negative_pairs])
#             contrastive_loss += contrastive_loss_fn(neg_emb1, neg_emb2, -torch.ones(neg_emb1.size(0)).to(neg_emb1.device))

#         # Combine classification and contrastive loss
#         total_loss = classification_loss + 0.1 * contrastive_loss  # Adjust weight as needed

#         return (total_loss, outputs) if return_outputs else total_loss


In [14]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=validation_dataset,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    num_train_epochs = 8,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    load_best_model_at_end=True,
    weight_decay=0.01,
    report_to = [],
    learning_rate = 8e-5,
    warmup_steps = 30,
    logging_steps= 50,
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
import matplotlib.pyplot as plt

# Extract the log history from the trainer
log_history = trainer.state.log_history

# Initialize lists for storing values
losses = []
accuracies = []
precisions = []
f1_scores = []
recalls = []

# Parse through the log history and extract values
for entry in log_history:
    if 'eval_loss' in entry and 'eval_accuracy' in entry and 'eval_precision' in entry and 'eval_recall' in entry and 'eval_f1' in entry:
        losses.append(entry['eval_loss'])
        accuracies.append(entry['eval_accuracy'])
        precisions.append(entry['eval_precision'])
        f1_scores.append(entry['eval_f1'])
        recalls.append(entry['eval_recall'])

# Define epochs (adjust based on your actual epoch range)
epochs = list(range(1, len(losses) + 1))

# Create a figure and axis for plotting
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot the first metric (loss) on the primary y-axis
ax1.plot(epochs, losses, color='tab:orange', label='Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

# Create a secondary y-axis for Accuracy
ax2 = ax1.twinx()
ax2.plot(epochs, accuracies, color='tab:blue', linestyle='--', label='Accuracy')
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Create a tertiary y-axis for Precision
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third y-axis
ax3.plot(epochs, precisions, color='tab:green', linestyle='-', label='Precision')
ax3.set_ylabel('Precision', color='tab:green')
ax3.tick_params(axis='y', labelcolor='tab:green')

# Create a fourth y-axis for Recall
ax4 = ax1.twinx()
ax4.spines['right'].set_position(('outward', 120))  # Offset the fourth y-axis
ax4.plot(epochs, recalls, color='tab:red', linestyle=':', label='Recall')
ax4.set_ylabel('Recall', color='tab:red')
ax4.tick_params(axis='y', labelcolor='tab:red')

# Create a fifth y-axis for F1-score
ax5 = ax1.twinx()
ax5.spines['right'].set_position(('outward', 180))  # Offset the fifth y-axis
ax5.plot(epochs, f1_scores, color='tab:purple', linestyle='-.', label='F1-Score')
ax5.set_ylabel('F1-Score', color='tab:purple')
ax5.tick_params(axis='y', labelcolor='tab:purple')

# Add legends for each line
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
lines4, labels4 = ax4.get_legend_handles_labels()
lines5, labels5 = ax5.get_legend_handles_labels()

ax2.legend(lines1 + lines2 + lines3 + lines4 + lines5,
           labels1 + labels2 + labels3 + labels4 + labels5,
           loc='lower left', bbox_to_anchor=(1.05, 1))

# Title and grid
plt.title('Validation Metrics vs Epochs')
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
model.save_pretrained("/content/")
model = BertForSequenceClassification.from_pretrained("/content/")

In [ ]:
test_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))
test_labels_df

In [ ]:
test_labels_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))
test_labels_df

In [ ]:
# predictions = []
# threshold = 0.5

# for idx, row in test_df.iterrows():
#     arg_id = row['Index']
#     text = row['Prompt']
#     encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
#     with torch.no_grad():
#         outputs = model(**encoding)

#     sigmoid = torch.nn.Sigmoid()
#     predictions_tensor = sigmoid(outputs.logits)

#     predicted_labels = []
#     for prob in predictions_tensor.flatten():
#         if prob >= threshold:
#           label = 1
#         else:
#           label =  0
#         predicted_labels.append(label)

#     predictions.append([arg_id] + predicted_labels)

# data = {'Index': [entry[0] for entry in predictions]}

# for i, label in enumerate(test_labels_df.columns[2:]):
#     data[label] = [entry[i+1] for entry in predictions]
# predictions_df = pd.DataFrame(data)


# # output_path = "/content/output/my_model_predictions.tsv"
# # predictions_df.to_csv(output_path, sep='\t', index=False)


In [ ]:
predictions = []
threshold = 0.51

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Check for GPU or use CPU
model.to(device)  # Move model to device

for idx, row in test_df.iterrows():
    arg_id = row['Index']
    text = row['Prompt']

    # Tokenize and move inputs to the same device
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    encoding = {key: value.to(device) for key, value in encoding.items()}

    with torch.no_grad():
        outputs = model(**encoding)

    sigmoid = torch.nn.Sigmoid()
    predictions_tensor = sigmoid(outputs.logits)  # Apply sigmoid for probabilities

    # Convert predictions to binary labels based on threshold
    predicted_labels = (predictions_tensor >= threshold).int().squeeze().tolist()  # Use tensor comparison

    predictions.append([arg_id] + predicted_labels)

# Prepare final DataFrame
data = {'Index': [entry[0] for entry in predictions]}

for i, label in enumerate(test_labels_df.columns[2:]):
    data[label] = [entry[i + 1] for entry in predictions]

predictions_df = pd.DataFrame(data)

# Uncomment to save the predictions to a file
# output_path = "/content/output/my_model_predictions.tsv"
# predictions_df.to_csv(output_path, sep='\t', index=False)


In [ ]:
predictions_df

In [ ]:
true_pred = test_labels_df[['Index', 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
true_pred

In [ ]:
true_labels = true_pred[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
pred_labels = predictions_df[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]

f1_average = f1_score(true_labels, pred_labels, average='macro')
print(f"Average F1 Score: {f1_average}")

In [ ]:

from sklearn.metrics import accuracy_score

true_labels = true_pred[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
pred_labels = predictions_df[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]

exact_match_count = 0
total_count = len(true_labels)

for i in range(total_count):
  if true_labels.iloc[i].equals(pred_labels.iloc[i]):
    exact_match_count += 1

exact_match_ratio = exact_match_count / total_count

print(f"Exact Match Ratio: {exact_match_ratio}")